In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

tf.random.set_seed(777)
print(tf.__version__)

2.3.0


In [2]:
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

log_path = "./logs/xor"
writer = tf.summary.create_file_writer(log_path)


In [3]:
W1 = tf.Variable(tf.random.normal((2, 10)), name='weight1')
b1 = tf.Variable(tf.random.normal((10,)), name='bias1')

W2 = tf.Variable(tf.random.normal((10, 10)), name='weight2')
b2 = tf.Variable(tf.random.normal((10,)), name='bias2')

W3 = tf.Variable(tf.random.normal((10, 10)), name='weight3')
b3 = tf.Variable(tf.random.normal((10,)), name='bias3')

W4 = tf.Variable(tf.random.normal((10, 1)), name='weight4')
b4 = tf.Variable(tf.random.normal((1,)), name='bias4')

def neural_net(features, step):
    layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)
    layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)
    layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)
    hypothesis = tf.sigmoid(tf.matmul(layer3, W4) + b4)

#     with tf.contrib.summary.record_summaries_every_n_global_steps(1):
#          tf.contrib.summary.histogram("weights1", W1)
#          tf.contrib.summary.histogram("biases1", b1)
#          tf.contrib.summary.histogram("layer1", layer1)
#         ….
#          tf.contrib.summary.histogram("weights3", W3)
#          tf.contrib.summary.histogram("biases3", b3)
#          tf.contrib.summary.histogram("layer3", layer3)
#          tf.contrib.summary.histogram("weights4", W4)
#          tf.contrib.summary.histogram("biases4", b4)
#          tf.contrib.summary.histogram("hypothesis", hypothesis)

    with writer.as_default():
        tf.summary.histogram("weights1", W1, step=step)
        tf.summary.histogram("biases1", b1, step=step)
        tf.summary.histogram("layer1", layer1, step=step)

        tf.summary.histogram("weights2", W2, step=step)
        tf.summary.histogram("biases2", b2, step=step)
        tf.summary.histogram("layer2", layer2, step=step)

        tf.summary.histogram("weights3", W3, step=step)
        tf.summary.histogram("biases3", b3, step=step)
        tf.summary.histogram("layer3", layer3, step=step)

        tf.summary.histogram("weights4", W4, step=step)
        tf.summary.histogram("biases4", b4, step=step)
        tf.summary.histogram("hypothesis", hypothesis, step=step)
    return hypothesis

In [4]:
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    with writer.as_default():
        tf.summary.scalar('loss', cost, step=step)
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

def grad(hypothesis, features, labels, step):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(neural_net(features, step),labels)
    return tape.gradient(loss_value, [W1, W2, W3, W4, b1, b2, b3, b4])


In [5]:
EPOCHS = 3000

for step in range(EPOCHS):    
    for features, labels  in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(neural_net(features, step), features, labels, step)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W1, W2, W3, W4, b1, b2, b3, b4]))
        if step % 50 == 0:
            loss_value = loss_fn(neural_net(features, step),labels)
            print("Iter: {}, Loss: {:.4f}".format(step, loss_value))
            
x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data, step),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

Iter: 0, Loss: 0.9517
Iter: 50, Loss: 0.6936
Iter: 100, Loss: 0.6923
Iter: 150, Loss: 0.6912
Iter: 200, Loss: 0.6901
Iter: 250, Loss: 0.6890
Iter: 300, Loss: 0.6879
Iter: 350, Loss: 0.6867
Iter: 400, Loss: 0.6855
Iter: 450, Loss: 0.6842
Iter: 500, Loss: 0.6827
Iter: 550, Loss: 0.6811
Iter: 600, Loss: 0.6793
Iter: 650, Loss: 0.6772
Iter: 700, Loss: 0.6749
Iter: 750, Loss: 0.6722
Iter: 800, Loss: 0.6690
Iter: 850, Loss: 0.6654
Iter: 900, Loss: 0.6611
Iter: 950, Loss: 0.6561
Iter: 1000, Loss: 0.6502
Iter: 1050, Loss: 0.6432
Iter: 1100, Loss: 0.6349
Iter: 1150, Loss: 0.6251
Iter: 1200, Loss: 0.6134
Iter: 1250, Loss: 0.5992
Iter: 1300, Loss: 0.5821
Iter: 1350, Loss: 0.5612
Iter: 1400, Loss: 0.5357
Iter: 1450, Loss: 0.5047
Iter: 1500, Loss: 0.4677
Iter: 1550, Loss: 0.4248
Iter: 1600, Loss: 0.3774
Iter: 1650, Loss: 0.3281
Iter: 1700, Loss: 0.2798
Iter: 1750, Loss: 0.2356
Iter: 1800, Loss: 0.1971
Iter: 1850, Loss: 0.1649
Iter: 1900, Loss: 0.1385
Iter: 1950, Loss: 0.1173
Iter: 2000, Loss: 0.100